In [2]:
import os
import json


# Path Evaluator

In [3]:
# Loading the pathEvaluator json which contain the data regarding the question and choices
with open('./data/pathEvaluator.json', 'r') as f:
    main_data = json.load(f)

In [4]:
# Question data subset of main_data containing only info about question
question_data = main_data['jsonGraph']['videos']['80988062']['interactiveVideoMoments']['value']["choicePointNavigatorMetadata"]["choicePointsMetadata"]["choicePoints"]  

# choice data subset of main_data containing only info about choices
choice_data = main_data['jsonGraph']['videos']['80988062']['interactiveVideoMoments']['value']["momentsBySegment"]

In [5]:
# bs_data relates parent_id, chocie_id, question, chocice text and default option
bs_data = {}
# stores the choice id and the corresponding text
choice_text = {}
# stores the question id and the corresponding text
# TODO: do it soon
# ques_text = {}

In [6]:
# returns the question for a given patent_id
# uses parent_if to find the question
# requires question data to be declared in the global scope
def find_ques(parent_id):
    return question_data[parent_id]["description"]
    
# example 
# print(find_ques(parent_id="3X"))

In [7]:
# add the relevant data (relates parent_id, chocie_id, question, chocice text and default option) to bs_data from choice_data
# arg : parent_id (id of the previous choice)
# expects question_data and choce_data is present in the global scope
# add the id and corresponding choice text to the choice_text 

def gen_data(parent_id):
    temp = {}
    # Some options may not have any Question 
    # example: single options choice are mostlikely not to have any question
    try:
        temp['question'] = find_ques(parent_id)
    except:
        # do nothing
        pass
        
    data = choice_data[parent_id]
    for items in data:
        if "choices" in items:
            temp["defaultChoiceIndex"] = items["defaultChoiceIndex"]
            # some choices don't have any text
            # example: there is a choice point where the user need to type the phone no 
            # correct phone no will be one choice and the wrong one will be the other.
            try:
                temp["choices"] = [{"id": item["id"], "text": item["text"], "segmentId": item["sg" if "sg" in item else "segmentId"]} for item in items["choices"]]
                
            except:
                # do something to add info about the phone no.
                # if ph no is available add else add xxx
                # temp[choice] = [{},{}]
                temp["choices"] = []
                for item in items["choices"]:
                    temp_dict = {}
                    temp_dict["id"] = item["id"]
                    if "segmentId" in item:
                        temp_dict["segmentId"] = item["segmentId"]
                    elif "sg" in item:
                        temp_dict["segmentId"] = item["sg"]
                    elif "trackingInfo" in items:
                        temp_dict["segmentId"] = item["trackingInfo"]["segmentId"]
                    if "text" in item:
                        temp_dict["text"] = item["text"]
                    elif "code" in item:
                        temp_dict["code"] = item["code"]
                    else:
                        temp_dict["code"] = "xxx"
                    temp["choices"].append(temp_dict)
            # adding the choice id and text to choice_text
            for choice in temp["choices"]:
                if choice["id"] not in choice_text:
                    choice_text[choice["segmentId"]] = choice['text' if "text" in choice else "code"]
                
    bs_data[parent_id]=temp


In [8]:
# returns the default choice for a given parent id
# expects the bs_data (created already) is available in the global scope
# bs_data relates parent_id, chocie_id, question, chocice text and default option

def find_default(parent_id):
    return bs_data[parent_id]["choices"][bs_data[parent_id]["defaultChoiceIndex"]]

    

In [9]:
def get_choice_text(choice_id):
    with open('./data/choice_text.json', 'r') as f:
        data = json.load(f)
    return(data[choice_id])

In [10]:
def get_choices_list(parent_id):
    with open('./data/bs_data.json', 'r') as f:
        data = json.load(f)
    return data[parent_id]["choices"]

# example
get_choices_list("1H")

[{'id': 'nsg-8AChoice', 'segmentId': '8AChoice', 'text': 'ACCEPT'},
 {'id': 'nsg-1Qnw-1Qtt', 'segmentId': '1Qnw-1Qtt', 'text': 'REFUSE'}]

In [11]:
# examples

# finds the question and the choice id with text for a given parent id 
# parent id is the id of the preious choice
# find_choices(parent_id="1A")

# print(find_default("1A"))
# find_ques(parent_id="2G")

In [12]:
# storing all id's
parent_ids=choice_data.keys();

In [13]:
# gen_data(list(parent_ids)[0])

In [14]:
# generating data for all the id's
for parent_id in parent_ids:
#     print(parent_id)
    gen_data(parent_id)
    
    
blank_keys = []
for key, data in bs_data.items():
    if len(data) == 0:
        blank_keys.append(key) 
for key in blank_keys:
    del bs_data[key]
len(bs_data)
    
# storing main data
with open('./data/bs_data.json', 'w') as json_file:
  json.dump(bs_data, json_file)

# saving the choice text
with open('./data/choice_text.json', 'w') as json_file:
  json.dump(choice_text, json_file)

# Data cl2

- first, find what is selected
- second, find if it was default or non-default

In [ ]:
# #  error

# def decode_cl2(cl2_path="./data/data_cl2.json"):
#     print('-'*10)
#     print("Processing : {}".format(cl2_path))
#     with open(cl2_path, 'r') as f:
#         data_cl2 = json.load(f)
    
#     selected_coice_sid = ""
#     parent_id = ""
#     reverseDeltas = data_cl2["reverseDeltas"]
    
#     for items in reverseDeltas:
#         for item in items:
#             if type(item) == dict and "view" in item:
#                 if item["view"] == "ikoChoicePointOption":
#                     selected_coice_sid = item["trackingInfo"]["segmentId"]
#                 elif item["view"] == "ikoChoicePoint":
#                     parent_id = item["trackingInfo"]["momentId"].split("_")[1]
#                     given_choices= item["trackingInfo"]["choices"]
    
#     if selected_coice_sid == "" and parent_id == "":
#         print("error: does not looks like a data cl2")
        
#     elif selected_coice_sid != "" and parent_id != "":
        
#         try:
#             print("Question Id: "+parent_id)
#             print("Question text: "+find_ques(parent_id))
#         except:
#             print("No Question present for this choice! ")
#             with open('./data/bs_data.json', 'r') as f:
#                 bs_data = json.load(f)
#             print("This question has {} options".format(len(bs_data[parent_id]["choices"])))
        
#         if len(given_choices) == 1:
#             print("fakeOption. Only {} options".format(len(given_choices)))
#         else:
        
#             try:
#                 print("Selected choice ID :"+ selected_coice_sid)
#                 print("Selected choice :"+get_choice_text(selected_coice_sid))
#             except:
#                 # May Crash Queck Fix.
#                 print("Opsee! error finding selected choice.. trying again!")
#                 found_choice = ""
#                 for choice in given_choices:
#                     if choice["segmentId"] == selected_coice_sid:
#                         selected_coice_id = choice["id"]
#                         for ch in get_choices_list(parent_id):
#                             if ch["id"] == selected_coice_id:
#                                 found_choice = ch["text" if "text" in ch else "code"]

#                 print("Selected choice :"+ found_choice)
            
#             print("Default chocie: "+str(find_default(parent_id)))
#     elif selected_coice_sid != "":
#         try:
#             print("Selected choice ID :"+ selected_coice_sid)
#             print("Selected choice :"+get_choice_text(selected_coice_sid))
#         except:
#             # May Crash Queck Fix.
#             print("Opsee! error finding selected choice")
#     print('-'*10)
    

In [ ]:
# decode_cl2("./data/cl2/1_cl2.json")

In [ ]:
# # renameing all files
# i = 0
# for filename in os.listdir("./data/cl2"): 
#         dst =str(i) + "_cl2.json"
#         src ='./data/cl2/'+ filename 
#         dst ='./data/cl2/'+ dst 
          
#         # rename() function will 
#         # rename all the files 
#         os.rename(src, dst) 
#         i += 1

In [ ]:
# for cl2 in os.listdir("./data/cl2"):
#     decode_cl2("./data/cl2/"+cl2)

In [15]:
selection_data = {}

def decode_cl2(cl2_path="./data/data_cl2.json"):
#     print('-'*10)
#     print("Processing : {}".format(cl2_path))
    with open(cl2_path, 'r') as f:
        data_cl2 = json.load(f)
    
    selected_coice_sid = ""
    parent_id = ""
    reverseDeltas = data_cl2["reverseDeltas"]
    currentState = data_cl2["currentState"]
    temp = {}
    
    for key, items in currentState.items():
        if "view" in items:
            if items["view"] == "ikoChoicePoint":
                parent_id = items["trackingInfo"]["momentId"].split("_")[1]
                given_choices= items["trackingInfo"]["choices"]
                break
    
    for items in reverseDeltas:
        for item in items:
            if type(item) == dict and "view" in item:
                if item["view"] == "ikoChoicePointOption":
                    selected_coice_sid = item["trackingInfo"]["segmentId"]
                elif item["view"] == "ikoChoicePoint":
                    parent_id = item["trackingInfo"]["momentId"].split("_")[1]
                    given_choices= item["trackingInfo"]["choices"]

        
    if selected_coice_sid != "" and parent_id != "":
        try:
            temp["id"] = parent_id 
            temp["question"] = find_ques(parent_id) 
            print("Question Id: {}\t text: {}".format(parent_id,find_ques(parent_id)))
        except:
            temp["error"] = "No queston present" 
            print("No Question present for this choice! ")
        
        if len(given_choices) == 1:
            print("fakeOption. Only {} options".format(len(given_choices)))
        else:
            try:
                print("Selected choice Id: {}\t text: {}".format(selected_coice_sid,get_choice_text(selected_coice_sid)))
                temp["sid"] = selected_coice_sid 
                temp["ctext"] = get_choice_text(selected_coice_sid) 
            except:
                # May Crash Queck Fix.
                print("Opsee! error finding selected choice.. trying again!")
                found_choice = ""
                for choice in given_choices:
                    if choice["segmentId"] == selected_coice_sid:
                        selected_coice_id = choice["id"]
                        for ch in get_choices_list(parent_id):
                            if ch["id"] == selected_coice_id:
                                found_choice = ch["text" if "text" in ch else "code"]
                print("Selected choice Id: {}\t text: {}".format(selected_coice_sid,found_choice))
                temp["sid"] = selected_coice_sid 
                temp["ctext"] = found_choice 
            
            print("Default chocie: "+str(find_default(parent_id)))
            temp["dchoice"] = find_default(parent_id)
            selection_data[parent_id]=temp
            print('-'*10)
            
#     elif selected_coice_sid != "":
#         print("only selected choice data found.")
#         try:
#             print("Selected choice Id: {}\t text: {}".format(selected_coice_sid,get_choice_text(selected_coice_sid)))
#         except:
#             print("Opsee! error finding selected choice. \t Selected choice Id: {}".format(selected_coice_sid))
#     print('-'*10)

In [16]:

for i in range(68):
    decode_cl2("./data/cl2/cl2({})".format(i))

Question Id: 1A	 text: Which Cereal?
Selected choice Id: 1D	 text: FROSTIES
Default chocie: {'id': '1E', 'text': 'SUGAR PUFFS', 'segmentId': '1E'}
----------
Question Id: 1D	 text: Which Tape?
Selected choice Id: 1G	 text: NOW 2
Default chocie: {'id': '1H', 'text': 'THOMPSON TWINS', 'segmentId': '1H'}
----------
Question Id: 1G	 text: Job Offer
Opsee! error finding selected choice.. trying again!
Selected choice Id: 1Qnw	 text: REFUSE
Default chocie: {'id': 'nsg-8AChoice', 'text': 'ACCEPT', 'segmentId': '8AChoice'}
----------
Question Id: 1Qnw	 text: Talk About Mum?
Opsee! error finding selected choice.. trying again!
Selected choice Id: 2B	 text: YES
Default chocie: {'id': 'nsg-2BChoice', 'text': 'YES', 'segmentId': '2BChoice'}
----------
Question Id: 1Qnw	 text: Talk About Mum?
Opsee! error finding selected choice.. trying again!
Selected choice Id: 2B	 text: YES
Default chocie: {'id': 'nsg-2BChoice', 'text': 'YES', 'segmentId': '2BChoice'}
----------
Question Id: 2B	 text: Which Rec

FileNotFoundError: [Errno 2] No such file or directory: './data/cl2/cl2(57)'

In [ ]:
print(json.dumps(selection_data, indent=2))

In [ ]:
with open('./data/user_choice.json', 'w') as json_file:
  json.dump(selection_data, json_file)